從 https://github.com/Yuliang-Liu/Box_Discretization_Network
下載ReCTS 2019 model，放到同樣的資料夾。

還有比賽的訓練和測試資料，也放到同樣的資料夾。

然後從頭執行到尾應該就能訓練邊預測結果了，
本程式碼未經測試，所以有可能會出錯，如果執行上產生問題請再跟我聯絡，謝謝。

# Import包包

In [ ]:
import os
import glob
import json
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from PIL import Image, ImageOps

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && python setup.py build_ext install
!git cd Box_Discretization_Network && python setup.py build develop

# 資料準備

## 準備private set預測格式

In [ ]:
data_list = []
for i in tqdm(range(3001, 5501)):
    img_path = f'img_{i}.jpg'

    d = {}
    d['img_name'] = img_path
    d['annotations'] = []
    
    data_list.append(d)

In [ ]:
train_data = {}
train_data['images'] = []

for i in tqdm(range(3001, 5501)):
    img_path = f'img_{i}.jpg'
    
    im = Image.open(f'PrivateTestDataset/img/{img_path}')
    im = ImageOps.exif_transpose(im)
    w, h = im.size
    im.save(f'PrivateTestDataset_exif/img/{img_path.lower()}')

    train_data['images'].append({
        "id":i,
        "file_name": img_path.lower(),
        "height": h,
        "width": w,
    })

In [ ]:
train_data['categories'] = [{
    "supercategory": "beverage",
    "id": 1,
    "name": "text"
}]

In [ ]:
with open('./PrivateTestDataset_exif/coco_test.json', 'w')as file:
    json.dump(train_data, file, indent=4)

## 準備訓練用資料格式

In [ ]:
def round_list(data):
    return np.array(data).round().astype('int').tolist()

In [ ]:
data_list = []
for i in tqdm(range(1, 4001)):
    img_path = f'img_{i}.jpg'
    label_path = f'TrainDataset/json/img_{i}.json'
    
    with open(label_path) as f:
        label = json.load(f)
    
    d = {}
    d['img_name'] = img_path
    d['annotations'] = []
    for points in label['shapes']:
        points = np.array(points['points']).astype('int').reshape(-1)
        xt = [points[ikpart] for ikpart in range(0, 8, 2)]
        yt = [points[ikpart] for ikpart in range(1, 8, 2)]
        xmin = min(xt)
        xmax = max(xt)
        ymin = min(yt)
        ymax = max(yt)
        d['annotations'].append([xmin, ymin, xmax, ymax, *points])
    
    
    data_list.append(d)

In [ ]:
for d in tqdm(data_list):
    filename = d['img_name']
    annotations = d['annotations']
    
    filename = filename.split('.')[0] + '.txt'
    with open(f'TrainDataset/bdn_label_file/{filename}', 'w') as file:
        for ann in annotations:
            file.write(','.join([str(a) for a in ann]) + '\n')

# 訓練模型

## 準備訓練用json檔

In [ ]:
!python convert_to_BDN_sequence_free.py TrainDataset aicup_train_bal 10
!python convert_to_BDN_sequence_free.py TrainDataset aicup_valid_bal 10

## 進行訓練

In [ ]:
!cd Box_Discretization_Network && sh quick_train_guide.sh

## 平均權重

In [ ]:
import torch

In [ ]:
ckps = []
for i in tqdm(range(12000, 20000+1, 1000)):
    ckp_path = f'Box_Discretization_Network/output/BDN/rects_ft/model_00{i:05d}.pth'
    ckps.append(torch.load(ckp_path, map_location='cpu'))

In [ ]:
new_ckp = torch.load(ckp_path, map_location='cpu')

for model_key in tqdm(new_ckp['model'].keys()):
    if 'int' in str(new_ckp['model'][model_key].dtype):
        print(f'Skip {model_key}')
        continue
    weights = [ckp['model'][model_key] for ckp in ckps]
    new_ckp['model'][model_key] = torch.mean(torch.stack(weights), 0)

In [ ]:
torch.save(
    new_ckp,
    'Box_Discretization_Network/output/BDN/rects_ft/model_avg.pth',
    _use_new_zipfile_serialization=False,
)

## 使用模型預測

In [ ]:
!cd Box_Discretization_Network && sh my_test.sh

# 轉換預測格式

## To Upload

In [ ]:
data_path = "Box_Discretization_Network/output/BDN/rects_ft_new_12kto20k_private_exif/inference/aicup_private_test_exif/kes.json"

with open(data_path, 'r') as file:
    raw_pred = json.load(file)
    
result = []
thres = 0.91
for p in raw_pred:
    if p['score'] > thres:
        result.append([p['image_id'], *np.array(p['ke_quad']).reshape(-1), 1.0])

In [ ]:
pred_df = pd.DataFrame(result)

In [ ]:
pred_df.to_csv('final.csv', index=False, header=False)